This is a second dataset to be used with UCS-Satellite-Database.

In [1]:
import pandas as pd

In [5]:
df_debris = pd.read_csv('../data/original/satcat.csv')

df_debris.head(10)

,OBJECT_NAME,OBJECT_ID,NORAD_CAT_ID,OBJECT_TYPE,OPS_STATUS_CODE,OWNER,LAUNCH_DATE,LAUNCH_SITE,DECAY_DATE,PERIOD,INCLINATION,APOGEE,PERIGEE,RCS,DATA_STATUS_CODE,ORBIT_CENTER,ORBIT_TYPE
0,SL-1 R/B,1957-001A,1,R/B,D,CIS,1957-10-04,TYMSC,1957-12-01,96.19,65.10,938.0,214.0,20.420,NaN,EA,IMP
1,SPUTNIK 1,1957-001B,2,PAY,D,CIS,1957-10-04,TYMSC,1958-01-03,96.10,65.00,1080.0,64.0,NaN,NaN,EA,IMP
2,SPUTNIK 2,1957-002A,3,PAY,D,CIS,1957-11-03,TYMSC,1958-04-14,103.74,65.33,1659.0,211.0,0.080,NaN,EA,IMP
3,EXPLORER 1,1958-001A,4,PAY,D,US,1958-02-01,AFETR,1970-03-31,88.48,33.15,215.0,183.0,NaN,NaN,EA,IMP
4,VANGUARD 1,1958-002B,5,PAY,NaN,US,1958-03-17,AFETR,NaN,132.60,34.25,3820.0,651.0,0.122,NaN,EA,ORB
5,EXPLORER 3,1958-003A,6,PAY,D,US,1958-03-26,AFETR,1958-06-28,151.74,19.24,5985.0,107.0,NaN,NaN,EA,IMP
6,SL-1 R/B,1958-004A,7,R/B,D,CIS,1958-05-15,TYMSC,1958-12-03,102.74,65.14,1571.0,206.0,NaN,NaN,EA,IMP
7,SPUTNIK 3,1958-004B,8,PAY,D,CIS,1958-05-15,TYMSC,1960-04-06,88.43,65.06,255.0,139.0,11.840,NaN,EA,IMP
8,EXPLORER 4,1958-005A,9,PAY,D,US,1958-07-26,AFETR,1959-10-23,92.81,50.25,585.0,239.0,NaN,NaN,EA,IMP
9,SCORE,1958-006A,10,PAY,D,US,1958-12-18,AFETR,1959-01-21,98.21,32.29,1187.0,159.0,NaN,NaN,EA,IMP
